# OCA FLAT Files Scan and Analysis

In [1]:
from concurrent.futures import ProcessPoolExecutor
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import AltAz, get_sun, EarthLocation
from tqdm.notebook import tqdm
import polars as pl
import numpy as np
from pathlib import Path
# import bottleneck as bn  # jeśli chcesz użyć bottleneck


## Config

In [2]:
base_dir = '~/data/fits'
telescopes = ['zb08', 'wk06', 'jk15']

tel = 'jk15'

observatory = EarthLocation.of_site('OCA')

In [11]:
# Zakładam że obserwatorium jest znane
# np.:
loc = EarthLocation.of_site('OCA')
lat = loc.lat.value
lon = loc.lon.value
height = loc.height.value


Directories

In [3]:
flats_dir = Path(base_dir).expanduser() / Path(tel) / Path('processed-ofp/flats')


In [4]:
filters = [f.name for f in flats_dir.glob('*')]
print('Recognized filters:', filters)

Recognized filters: ['empty', 'i', 'z', 'Ha_w', 'u_s', 'Ic', 'b_s', 'B', 'V', 'g', 'v_s', 'y_s', 'r', 'u']


In [12]:
def process_file(p, f, lat, lon, height, tel):
    # Tworzymy EarthLocation na miejscu
    observatory = EarthLocation.from_geodetic(lon=lon, lat=lat, height=height)

    with fits.open(p) as hdul:
        hdu = hdul[0]
        date_obs = Time(hdu.header['DATE-OBS'], format='isot', scale='utc')
        altaz_frame = AltAz(obstime=date_obs, location=observatory)
        data = hdu.data
        lower, upper = np.percentile(data, [5, 95])

        mean_central_90 = np.mean(data[(data > lower) & (data < upper)]) if np.any((data > lower) & (data < upper)) else np.nan

        return {
            'filter': f,
            'file': p.name,
            'exptime': float(hdu.header['EXPTIME']),
            'date-obs': date_obs.to_datetime(),
            'sun-alt': float(get_sun(date_obs).transform_to(altaz_frame).alt.deg),
            'mean': float(data.mean()),
            'median': float(np.median(data)),
            'std': float(data.std()),
            'min': float(data.min()),
            'max': float(data.max()),
            'min-05': float(lower),
            'max-95': float(upper),
            'mean-central-90': float(mean_central_90),
            'read-mod': int(hdu.header['READ-MOD']),
            'gain-mod': int(hdu.header['GAIN-MOD']),
            'flat-era': int(hdu.header.get('FLAT-ERA', 0)),
            'test': int(hdu.header.get('TEST', 0)),
            'bitpix': int(hdu.header['BITPIX']),
        }


In [13]:
all_paths = []
for f in filters:
    paths = list((flats_dir / Path(f)).glob(f'*/{tel}?_????_?????.fits'))
    for p in paths:
        all_paths.append((p, f))

In [14]:
print(f'Found {len(all_paths)} FITS files')

Found 1225 FITS files


In [15]:
stats_df = None


In [16]:
def process_file_wrapper(args):
    p, f = args
    return process_file(p, f, lat, lon, height, tel)

with ProcessPoolExecutor() as executor:
    results = list(tqdm(
        executor.map(process_file_wrapper, all_paths),
        total=len(all_paths),
        desc="Processing FITS"
    ))

Processing FITS:   0%|          | 0/1225 [00:00<?, ?it/s]

Process SpawnProcess-21:
Process SpawnProcess-18:
Process SpawnProcess-19:
Traceback (most recent call last):
Process SpawnProcess-17:
Process SpawnProcess-20:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/concurrent/futures/process.py", line 244, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeEr

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [31]:
stats_df = pl.DataFrame(results)
print(stats_df)

(<Longitude -70.20128 deg>, <Latitude -24.59867 deg>, <Quantity 2817. m>)

In [21]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import polars as pl
from pathlib import Path

# Wczytywanie plików FITS
def process_file(p, f, lat, lon, height, tel):
    observatory = EarthLocation.from_geodetic(lon=lon, lat=lat, height=height)
    try:
        with fits.open(p) as hdul:
            hdu = hdul[0]
            date_obs = Time(hdu.header['DATE-OBS'], format='isot', scale='utc')
            altaz_frame = AltAz(obstime=date_obs, location=observatory)
            data = hdu.data

            lower, upper = np.percentile(data, [5, 95])
            mean_central_90 = np.mean(data[(data > lower) & (data < upper)]) if np.any((data > lower) & (data < upper)) else np.nan

            return {
                'filter': f,
                'file': p.name,
                'exptime': float(hdu.header['EXPTIME']),
                'date-obs': date_obs.to_datetime(),
                'sun-alt': float(get_sun(date_obs).transform_to(altaz_frame).alt.deg),
                'mean': float(data.mean()),
                'median': float(np.median(data)),
                'std': float(data.std()),
                'min': float(data.min()),
                'max': float(data.max()),
                'min-05': float(lower),
                'max-95': float(upper),
                'mean-central-90': float(mean_central_90),
                'read-mod': int(hdu.header['READ-MOD']),
                'gain-mod': int(hdu.header['GAIN-MOD']),
                'flat-era': int(hdu.header.get('FLAT-ERA', 0)),
                'test': int(hdu.header.get('TEST', 0)),
                'bitpix': int(hdu.header['BITPIX']),
            }
    except Exception as e:
        return {'error': str(e), 'file': p.name}

# Inicjalizacja zmiennych
base_dir = '~/data/fits'
tel = 'jk15'
loc = EarthLocation.of_site('OCA')
lat, lon, height = loc.lat.value, loc.lon.value, loc.height.value

flats_dir = Path(base_dir).expanduser() / Path(tel) / Path('processed-ofp/flats')
filters = [f.name for f in flats_dir.glob('*')]

# Zbieranie ścieżek
all_paths = [(p, f, lat, lon, height, tel)
             for f in filters
             for p in (flats_dir / Path(f)).glob(f'*/{tel}?_????_?????.fits')]

# Przetwarzanie równoległe za pomocą ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    results = list(tqdm(
        executor.map(lambda args: process_file(*args), all_paths),
        total=len(all_paths),
        desc="Processing FITS"
    ))

# Tworzenie DataFrame
stats_df = pl.DataFrame(results)
print(stats_df)

Processing FITS:   0%|          | 0/1225 [00:00<?, ?it/s]

shape: (1_225, 18)
┌────────┬─────────────────┬───────────┬─────────────────┬───┬──────────┬──────────┬──────┬────────┐
│ filter ┆ file            ┆ exptime   ┆ date-obs        ┆ … ┆ gain-mod ┆ flat-era ┆ test ┆ bitpix │
│ ---    ┆ ---             ┆ ---       ┆ ---             ┆   ┆ ---      ┆ ---      ┆ ---  ┆ ---    │
│ str    ┆ str             ┆ f64       ┆ datetime[μs]    ┆   ┆ i64      ┆ i64      ┆ i64  ┆ i64    │
╞════════╪═════════════════╪═══════════╪═════════════════╪═══╪══════════╪══════════╪══════╪════════╡
│ empty  ┆ jk15c_0639_6362 ┆ 0.5       ┆ 2024-11-25      ┆ … ┆ 2        ┆ 0        ┆ 0    ┆ 16     │
│        ┆ 5.fits          ┆           ┆ 03:16:11.956925 ┆   ┆          ┆          ┆      ┆        │
│ empty  ┆ jk15c_0639_6401 ┆ 0.5       ┆ 2024-11-25      ┆ … ┆ 2        ┆ 0        ┆ 0    ┆ 16     │
│        ┆ 6.fits          ┆           ┆ 03:21:49.651920 ┆   ┆          ┆          ┆      ┆        │
│ empty  ┆ jk15c_0639_6326 ┆ 0.5       ┆ 2024-11-25      ┆ … ┆ 2        